## 1. Obtenemos la siguientes medidas:

### Obtenemos las siguientes métricas:
- P(e|d)
- P(d|e)
- ICC
- TS


In [1]:
import os
import pickle
import math

In [2]:
def deserialize_object(path):
    pickle_in = open(path,"rb")
    obj = pickle.load(pickle_in)
    pickle_in.close()
    print("Cargado el objeto", path.split("/")[- 1])
    return obj

In [3]:
path_dic = 'dic_specialties.pkl'
dic_spe = deserialize_object(path_dic)

Cargado el objeto dic_specialties.pkl


In [4]:
#Nº de documentos en la especialidad que NO tienen el n-gram
def obtain_ndep(list_doc_term, list_doc_total_spe):
    return len(set(set(list_doc_total_spe)-set(list_doc_term)))

In [5]:
# Et: nº de especialidades que contienen el ngram
def obtain_et(term, dic_spe):
    et = 0

    for specialty, dic in dic_spe.items():
        
        # obtenemos el diccionario de términos
        dic_term = dic['terms']
        
         # si el término se encuentra en el diccionario sumamos 1
        if term in dic_term:
            et += 1
            
    return et      

In [6]:
#Número de documentos en las OTRAS especialidades que tienen el n-gram
def obtain_ndpe(term, dic_spe, specialty_actual):
    
    ndpe = 0

    for specialty, dic in dic_spe.items():
        
        # si la especialidad no es la misma
        if not specialty == specialty_actual:
            
            # obtenemos el diccionario de términos
            dic_term = dic['terms']
            
            # si el término se encuentra en el diccionario
            if term in dic_term:
                
                # obtenemos los documentos en los que aparece
                doc_term = list(set(dic_term[term]))
                
                # contamos el nº de documentos y sumamos
                ndpe += len(doc_term)
                
    return ndpe

In [7]:
# obtenemos las veces que se repite el termino en el corpus
def obtain_nocc_term(term, dic_spe):
    
    nocc_term = 0

    for specialty, dic in dic_spe.items():
        # obtenemos el diccionario de términos
        dic_term = dic['terms']

        # si el término se encuentra en el diccionario
        if term in dic_term:

            # obtenemos el nº los documentos en los que aparece
            nocc_term += len(dic_term[term])
                
    return nocc_term

In [8]:
# t-score
#Nº de palabras en el corpus: 6469046
#Nº de palabras diferentes en el corpus: 115121
N = 6469046
def obtain_tscore(term, dic_spe):
    
    # unigramas
    if not type(term) is tuple:
        nocc_term = obtain_nocc_term(term, dic_spe)
        
        ts = nocc_term - nocc_term / math.sqrt(nocc_term / N)
        
        return ts, nocc_term, 0, 0, 0
       
    else:
        # brigramas
        if len(term) == 2:
            nocc_bigram = obtain_nocc_term(term, dic_spe)
            nocc_w1 = obtain_nocc_term(term[0], dic_spe)
            nocc_w2 = obtain_nocc_term(term[1], dic_spe)
        
            ts = nocc_bigram - (nocc_w1 * nocc_w2) / math.sqrt(nocc_bigram / N)
            
            return ts, nocc_bigram, nocc_w1, nocc_w2, 0
            
        # trigramas
        elif len(term) == 3:
            
            nocc_trigram = obtain_nocc_term(term, dic_spe)
            nocc_w1 = obtain_nocc_term(term[0], dic_spe)
            nocc_w2 = obtain_nocc_term(term[1], dic_spe)
            nocc_w3 = obtain_nocc_term(term[2], dic_spe)
            
            ts =  ( nocc_trigram - (nocc_w1 * nocc_w2 * nocc_w3) ) / math.sqrt(nocc_trigram / N)
        
            return ts, nocc_trigram, nocc_w1, nocc_w2, nocc_w3

In [10]:
def calculate_weight_terms(dic_spe):
    
    list_spe_import = ['H02.403.680.600_rehabilitation', 'H02.403.225_dermatology', 'H02.403.763.750_gynecology', 'H02.403.429.163_cardiology', 'H02.403.740_radiology']
    for specialty, dic in dic_spe.items():
        
        if specialty in list_spe_import:
            
            with open('./term_weight/' +specialty + "_terms_weight.csv", "w") as fout:
                fout.write("Término|")
                
                fout.write("Nº occ esp|")
                
                fout.write("N(d,e)|")
                fout.write("N(d,e')|")
                fout.write("N(d',e)|")
                
                fout.write("Et|")
                
                fout.write("P(e,d)|")
                fout.write("P(d,e)|")
                
                fout.write("ICC|")
                fout.write("T-score|")
                
                fout.write("P(w1,w2,w3)|")
                fout.write("P(w1)|")
                fout.write("P(w3)|")
                fout.write("P(w2)|")
                
                fout.write("CorpusWeight|")
                fout.write("SpecialtyWeight|")
                
                fout.write("TotalWeight|")
                fout.write("ICC*SpecialtyWeight|")
                
                fout.write("\n")
                
                list_doc_total_specialty = dic['docs']
                dic_terms = dic['terms']

                print("Especialidad:", specialty, 'nº docs total:', len(list_doc_total_specialty))
                    
                for term, list_doc_term in dic_terms.items():
                    
                    num_ooc_spe = len(list_doc_term)
                    
                    nde = len(set(list_doc_term))
        
                    ndep = obtain_ndep(list_doc_term, list_doc_total_specialty)
                    ndpe = obtain_ndpe(term, dic_spe, specialty)
                    et = obtain_et(term, dic_spe)
            
                    # p(e|d)
                    ped = nde / (nde + ndep)
                    
                    # p(d|e)
                    pde = nde / (nde + ndpe)
                   
                    # icc
                    icc = 1 / ( et + 1) # es el nº de otras especialidades más la q estamos leyendo

                    # t-score
                    tscore, nocc_term, nocc_w1, nocc_w2, nocc_w3 = obtain_tscore(term, dic_spe)
                    
                    # corpus level weight
                    clw = icc * tscore
                    
                    # specialty level weight
                    slw = ped + pde
                    
                    # total weight
                    tw = clw * slw
                    
                    # otra prueba (icc x specialty level weight)
                    other_total = icc * slw
                    
                    
                    # imprimir en fichero
                    fout.write(str(term) + "|")
                    
                    fout.write(str(num_ooc_spe) + "|")
                    
                    fout.write(str(nde) +'|')
                    fout.write(str(ndep)+ '|')
                    fout.write(str(ndpe) + '|')
                    
                    fout.write(str(et) + '|')
                    
                    fout.write(str(ped)+ '|')
                    fout.write(str(pde)+ '|')
                    
                    fout.write(str(icc)+ '|')
                    fout.write(str(tscore)+ '|')
                    
                    fout.write(str(nocc_term)+ '|')
                    fout.write(str(nocc_w1)+ '|')
                    fout.write(str(nocc_w2)+ '|')
                    fout.write(str(nocc_w3)+ '|')
                    
                    fout.write(str(clw) + "|")
                    fout.write(str(slw) + "|")
                
                    fout.write(str(tw) + "|")
                    fout.write(str(other_total) + "|")
                
                    fout.write('\n')
                
calculate_weight_terms(dic_spe)    

Especialidad: H02.403.763.750_gynecology nº docs total: 7732
Especialidad: H02.403.225_dermatology nº docs total: 9411
Especialidad: H02.403.429.163_cardiology nº docs total: 27316
Especialidad: H02.403.740_radiology nº docs total: 2062
Especialidad: H02.403.680.600_rehabilitation nº docs total: 2855
